In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('nlp').getOrCreate()

In [0]:
from pyspark.ml.feature import Tokenizer,RegexTokenizer

In [0]:
from pyspark.sql.functions import col,udf
from pyspark.sql.types import IntegerType

In [0]:
sentence_df = spark.createDataFrame([
  (0, 'Hello everyone and welcome to the tools for natural language processing lecture part 1.')
   ,(1, 'Before we jump straight into the code along project I want to take a brief moment to explore a few of the tools that SPARC has for dealing with text data understanding these tools.')
   ,(2, 'Were going to be able to use them easily in our custom coatl project.')
   ,(3, 'So were going to learn a lot of these basic features that youll find yourself using all the time.')
   ,(4, 'If,you,end,up,dealing,with,texts,they,show,a,spark,and,Python.')
],['id','sentence'])

In [0]:
sentence_df.show()

+---+--------------------+
 id| sentence|
+---+--------------------+
 0|Hello everyone an...|
 1|Before we jump st...|
 2|Were going to be ...|
 3|So were going to ...|
 4|If,you,end,up,dea...|
+---+--------------------+

In [0]:
tokenizer = Tokenizer(inputCol='sentence',outputCol='words')

In [0]:
regex_tokenizer = RegexTokenizer(inputCol='sentence',outputCol='words',pattern='\\W')

In [0]:
count_tokens=udf(lambda words:len(words),IntegerType())

In [0]:
df_tokenized = tokenizer.transform(sentence_df)

In [0]:
df_tokenized.show()

+---+--------------------+--------------------+
 id| sentence| words|
+---+--------------------+--------------------+
 0|Hello everyone an...|[hello, everyone,...|
 1|Before we jump st...|[before, we, jump...|
 2|Were going to be ...|[were, going, to,...|
 3|So were going to ...|[so, were, going,...|
 4|If,you,end,up,dea...|[if,you,end,up,de...|
+---+--------------------+--------------------+

In [0]:
df_tokenized.withColumn('tokens',count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
 id| sentence| words|tokens|
+---+--------------------+--------------------+------+
 0|Hello everyone an...|[hello, everyone,...| 14|
 1|Before we jump st...|[before, we, jump...| 34|
 2|Were going to be ...|[were, going, to,...| 14|
 3|So were going to ...|[so, were, going,...| 19|
 4|If,you,end,up,dea...|[if,you,end,up,de...| 1|
+---+--------------------+--------------------+------+

In [0]:
regex_tokenized = regex_tokenizer.transform(sentence_df)

In [0]:
regex_tokenized.withColumn('tokens',count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
 id| sentence| words|tokens|
+---+--------------------+--------------------+------+
 0|Hello everyone an...|[hello, everyone,...| 14|
 1|Before we jump st...|[before, we, jump...| 34|
 2|Were going to be ...|[were, going, to,...| 14|
 3|So were going to ...|[so, were, going,...| 19|
 4|If,you,end,up,dea...|[if, you, end, up...| 13|
+---+--------------------+--------------------+------+

In [0]:
from pyspark.ml.feature import StopWordsRemover

In [0]:
sentence_df2 = spark.createDataFrame([
  (0, ['Hello','everyone','and','welcome to the tools for natural language processing lecture part 1.'])
   ,(1, ['Before','we','jump','straight into the code along project I want to take a brief moment to explore a few of the tools that SPARC has for dealing with text data understanding these tools.'])
   ,(2, ['Were','going','to','be able to use them easily in our custom coatl project.'])
   ,(3, ['So','were','going','to learn a lot of these basic features that youll find yourself using all the time.'])
   ,(4, ['If','you','end','up','dealing','with','texts','they','show','a','spark','and','Python.'])
],['id','tokens'])

In [0]:
sentence_df2.show()

+---+--------------------+
 id| tokens|
+---+--------------------+
 0|[Hello, everyone,...|
 1|[Before, we, jump...|
 2|[Were, going, to,...|
 3|[So, were, going,...|
 4|[If, you, end, up...|
+---+--------------------+

In [0]:
remover = StopWordsRemover(inputCol='tokens',outputCol='filtered')

In [0]:
remover.transform(sentence_df2).show()

+---+--------------------+--------------------+
 id| tokens| filtered|
+---+--------------------+--------------------+
 0|[Hello, everyone,...|[Hello, everyone,...|
 1|[Before, we, jump...|[jump, straight i...|
 2|[Were, going, to,...|[going, be able t...|
 3|[So, were, going,...|[going, to learn ...|
 4|[If, you, end, up...|[end, dealing, te...|
+---+--------------------+--------------------+

In [0]:
from pyspark.ml.feature import NGram

In [0]:
word_df = spark.createDataFrame([
  (0, ['Hello','everyone','and','welcome to the tools for natural language processing lecture part 1.'])
   ,(1, ['Before','we','jump','straight into the code along project I want to take a brief moment to explore a few of the tools that SPARC has for dealing with text data understanding these tools.'])
   ,(2, ['Were','going','to','be able to use them easily in our custom coatl project.'])
   ,(3, ['So','were','going','to learn a lot of these basic features that youll find yourself using all the time.'])
   ,(4, ['If','you','end','up','dealing','with','texts','they','show','a','spark','and','Python.'])
],['id','words'])

In [0]:
ngram = NGram(n=2,inputCol='words',outputCol='grams')

In [0]:
ngram.transform(word_df).show()

+---+--------------------+--------------------+
 id| words| grams|
+---+--------------------+--------------------+
 0|[Hello, everyone,...|[Hello everyone, ...|
 1|[Before, we, jump...|[Before we, we ju...|
 2|[Were, going, to,...|[Were going, goin...|
 3|[So, were, going,...|[So were, were go...|
 4|[If, you, end, up...|[If you, you end,...|
+---+--------------------+--------------------+

In [0]:
ngram.transform(word_df).select('grams').show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
grams |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
[Hello everyone, everyone and, and welcome to the tools for natural language processing lecture part 1.] |
[Before we, we jump, jump straight into the code along project I want to take a brief moment to explore a few of the tools that SPARC has for dealing with text data understanding these tools.]|
[Were going, going to, to be able to use them easily in our custom coatl project.] |
[So were, were going, going to learn a lot of these basic features that youll find yourself using all the time.] |
[If you, you end, end up, up dealing, dealing with, with texts, texts they, they show, show a, a spark, spark and, and Python.] |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

In [0]:
from pyspark.ml.feature import HashingTF,IDF,Tokenizer

In [0]:
df_tokenized.show(truncate=False)

+---+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
id |sentence |words |
+---+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
0 |Hello everyone and welcome to the tools for natural language processing lecture part 1. |[hello, everyone, and, welcome, to, the, tools, for, natural, language, processing, lecture, part, 1.] |
1 |Before we jump straight into the code along project I want to take a brief moment to explore a few of the tools that SPARC has for dealing with text data understanding these tools.|[before, we, jump, straight, into, the, code, along, project, i, want, to, take, a, brief, moment, to, explore, a, few, of, the, tools, that, sparc, has, for, dealing, with, text, data, understanding, these, tools.]|
2 |Were going to be able to use them easily in our custom coatl project. |[were, going, to, be, able, to, use, them, easily, in, our, custom, coatl, project.] |
3 |So were going to learn a lot of these basic features that youll find yourself using all the time. |[so, were, going, to, learn, a, lot, of, these, basic, features, that, youll, find, yourself, using, all, the, time.] |
4 |If,you,end,up,dealing,with,texts,they,show,a,spark,and,Python. |[if,you,end,up,dealing,with,texts,they,show,a,spark,and,python.] |
+---+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

In [0]:
hashing_tf = HashingTF(inputCol='words',outputCol='rawFeatures')

In [0]:
df_featurized = hashing_tf.transform(df_tokenized)

In [0]:
idf = IDF(inputCol='rawFeatures',outputCol='features')

In [0]:
idf_model = idf.fit(df_featurized)

In [0]:
df_rescaled = idf_model.transform(df_featurized)

In [0]:
df_rescaled.select('id','features').show()

+---+--------------------+
 id| features|
+---+--------------------+
 0|(262144,[8804,275...|
 1|(262144,[12472,19...|
 2|(262144,[4629,275...|
 3|(262144,[22575,27...|
 4|(262144,[186633],...|
+---+--------------------+

In [0]:
from pyspark.ml.feature import CountVectorizer

In [0]:
df = spark.createDataFrame([
  (0,'a b c'.split(' '))
  ,(1, 'a b b c a'.split(' '))
], ['id', 'words'])

In [0]:
cv = CountVectorizer(inputCol='words',outputCol='features',vocabSize=3,minDF=2.0)

In [0]:
cv_model = cv.fit(df)

In [0]:
results = cv_model.transform(df)

In [0]:
results.show(truncate=False)

+---+---------------+-------------------------+
id |words |features |
+---+---------------+-------------------------+
0 |[a, b, c] |(3,[0,1,2],[1.0,1.0,1.0])|
1 |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

In [0]:
df_spam = spark.read.csv('/FileStore/tables/SMSSpamCollection',inferSchema=True,sep='\t')

In [0]:
df_spam.show()

+----+--------------------+
 _c0| _c1|
+----+--------------------+
 ham|Go until jurong p...|
 ham|Ok lar... Joking ...|
spam|Free entry in 2 a...|
 ham|U dun say so earl...|
 ham|Nah I don't think...|
spam|FreeMsg Hey there...|
 ham|Even my brother i...|
 ham|As per your reque...|
spam|WINNER!! As a val...|
spam|Had your mobile 1...|
 ham|I'm gonna be home...|
spam|SIX chances to wi...|
spam|URGENT! You have ...|
 ham|I've been searchi...|
 ham|I HAVE A DATE ON ...|
spam|XXXMobileMovieClu...|
 ham|Oh k...i'm watchi...|
 ham|Eh u remember how...|
 ham|Fine if thats th...|
spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows

In [0]:
df_spam = df_spam.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [0]:
from pyspark.sql.functions import length

In [0]:
df_spam = df_spam.withColumn('length',length(df_spam['text']))

In [0]:
df_spam.show()

+-----+--------------------+------+
class| text|length|
+-----+--------------------+------+
 ham|Go until jurong p...| 111|
 ham|Ok lar... Joking ...| 29|
 spam|Free entry in 2 a...| 155|
 ham|U dun say so earl...| 49|
 ham|Nah I don't think...| 61|
 spam|FreeMsg Hey there...| 147|
 ham|Even my brother i...| 77|
 ham|As per your reque...| 160|
 spam|WINNER!! As a val...| 157|
 spam|Had your mobile 1...| 154|
 ham|I'm gonna be home...| 109|
 spam|SIX chances to wi...| 136|
 spam|URGENT! You have ...| 155|
 ham|I've been searchi...| 196|
 ham|I HAVE A DATE ON ...| 35|
 spam|XXXMobileMovieClu...| 149|
 ham|Oh k...i'm watchi...| 26|
 ham|Eh u remember how...| 81|
 ham|Fine if thats th...| 56|
 spam|England v Macedon...| 155|
+-----+--------------------+------+
only showing top 20 rows

In [0]:
df_spam.groupBy('class').mean().show()

+-----+-----------------+
class| avg(length)|
+-----+-----------------+
 ham| 71.4545266210897|
 spam|138.6706827309237|
+-----+-----------------+

In [0]:
from pyspark.ml.feature import StringIndexer

In [0]:
spam_tokenizer = Tokenizer(inputCol='text',outputCol='token_text')
spam_stop_remover = StopWordsRemover(inputCol='token_text',outputCol='stop_token')
spam_count_vectorizer = CountVectorizer(inputCol='stop_token',outputCol='count_vec')
idf = IDF(inputCol='count_vec',outputCol='tf_idf')
ham_spam_to_numeric = StringIndexer(inputCol='class',outputCol='label')

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
data_cleanser = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

In [0]:
from pyspark.ml.classification import NaiveBayes

In [0]:
nb = NaiveBayes()

In [0]:
from pyspark.ml import Pipeline

In [0]:
data_preprocessing_pipeline = Pipeline(stages=[ham_spam_to_numeric
                                              ,spam_tokenizer
                                              ,spam_stop_remover
                                              ,spam_count_vectorizer
                                              ,idf
                                              ,data_cleanser])

In [0]:
job_clean = data_preprocessing_pipeline.fit(df_spam)

In [0]:
df_clean = job_clean.transform(df_spam)

In [0]:
df_clean.columns

Out[94]: ['class',
 'text',
 'length',
 'label',
 'token_text',
 'stop_token',
 'count_vec',
 'tf_idf',
 'features']

In [0]:
df_clean = df_clean.select('label','features')

In [0]:
df_clean.show()

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0|(13424,[7,11,31,6...|
 0.0|(13424,[0,24,297,...|
 1.0|(13424,[2,13,19,3...|
 0.0|(13424,[0,70,80,1...|
 0.0|(13424,[36,134,31...|
 1.0|(13424,[10,60,139...|
 0.0|(13424,[10,53,103...|
 0.0|(13424,[125,184,4...|
 1.0|(13424,[1,47,118,...|
 1.0|(13424,[0,1,13,27...|
 0.0|(13424,[18,43,120...|
 1.0|(13424,[8,17,37,8...|
 1.0|(13424,[13,30,47,...|
 0.0|(13424,[39,96,217...|
 0.0|(13424,[552,1697,...|
 1.0|(13424,[30,109,11...|
 0.0|(13424,[82,214,47...|
 0.0|(13424,[0,2,49,13...|
 0.0|(13424,[0,74,105,...|
 1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows

In [0]:
train_df,test_df = df_clean.randomSplit([0.7,0.3])

In [0]:
spam_detector = nb.fit(train_df)

In [0]:
df_spam.printSchema()

root
-- class: string (nullable = true)
-- text: string (nullable = true)
-- length: integer (nullable = true)

In [0]:
test_results = spam_detector.transform(test_df)

In [0]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
label| features| rawPrediction| probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
 0.0|(13424,[0,1,2,13,...|[-610.64402121776...|[0.99999999999999...| 0.0|
 0.0|(13424,[0,1,3,9,1...|[-577.41011841982...|[1.0,3.4298591055...| 0.0|
 0.0|(13424,[0,1,7,8,1...|[-912.56506655591...|[1.0,2.8718785120...| 0.0|
 0.0|(13424,[0,1,9,14,...|[-542.78345579467...|[1.0,3.7677554821...| 0.0|
 0.0|(13424,[0,1,17,19...|[-804.08122201970...|[1.0,4.5643652979...| 0.0|
 0.0|(13424,[0,1,21,27...|[-990.80352925121...|[1.0,3.8177478328...| 0.0|
 0.0|(13424,[0,1,23,63...|[-1293.8536015231...|[1.0,8.1060440999...| 0.0|
 0.0|(13424,[0,1,27,35...|[-1475.7789744973...|[0.99999999999285...| 0.0|
 0.0|(13424,[0,1,31,43...|[-342.68966845997...|[1.0,1.7364220969...| 0.0|
 0.0|(13424,[0,1,46,17...|[-1138.0717483741...|[4.63873745803325...| 1.0|
 0.0|(13424,[0,1,498,5...|[-321.18890492907...|[0.99999999993152...| 0.0|
 0.0|(13424,[0,1,874,1...|[-96.183646861454...|[0.99999995752067...| 0.0|
 0.0|(13424,[0,2,3,5,3...|[-488.85304736416...|[1.0,4.9382028052...| 0.0|
 0.0|(13424,[0,2,3,6,9...|[-3416.8781205468...|[1.0,8.0304169335...| 0.0|
 0.0|(13424,[0,2,3,6,9...|[-3416.8781205468...|[1.0,8.0304169335...| 0.0|
 0.0|(13424,[0,2,4,5,7...|[-986.17963327391...|[1.0,6.4081594782...| 0.0|
 0.0|(13424,[0,2,4,8,1...|[-1311.5297568381...|[1.0,1.9289706901...| 0.0|
 0.0|(13424,[0,2,4,8,2...|[-1398.7988459465...|[1.0,2.6402837783...| 0.0|
 0.0|(13424,[0,2,4,10,...|[-1225.6506722966...|[1.0,2.1720650559...| 0.0|
 0.0|(13424,[0,2,4,40,...|[-1578.6485314210...|[0.99999999868168...| 0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 20 rows

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [0]:
acc_eval = MulticlassClassificationEvaluator()

In [0]:
acc_nb = acc_eval.evaluate(test_results)

In [0]:
print('Accuracy of Naive Bayes Model')
print(acc_nb)

Accuracy of Naive Bayes Model
0.9178034878037432

In [0]:
from pyspark.ml.classification import LogisticRegression,DecisionTreeClassifier,RandomForestClassifier,GBTClassifier

In [0]:
logreg = LogisticRegression()
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier()
gbt = GBTClassifier()

In [0]:
spam_detector_logreg = logreg.fit(train_df)

In [0]:
logreg_test_results = spam_detector_logreg.transform(test_df)

In [0]:
acc_logreg = acc_eval.evaluate(logreg_test_results)

In [0]:
print('Accuracy of Logistic Regression Model')
print(acc_logreg)

Accuracy of Naive Bayes Model
0.9592005961379483

In [0]:
spam_detector_dtc = dtc.fit(train_df)

dtc_test_results = spam_detector_dtc.transform(test_df)

acc_dtc = acc_eval.evaluate(dtc_test_results)

print('Accuracy of Decision Tree Model')
print(acc_dtc)

Accuracy of Decision Tree Model
0.9395635241015619

In [0]:
spam_detector_rfc = rfc.fit(train_df)

rfc_test_results = spam_detector_rfc.transform(test_df)

acc_rfc = acc_eval.evaluate(rfc_test_results)

print('Accuracy of Random Forest Model')
print(acc_rfc)

Accuracy of Random Forest Model
0.8195175406127928

In [0]:
spam_detector_gbt = gbt.fit(train_df)

gbt_test_results = spam_detector_gbt.transform(test_df)

acc_gbt = acc_eval.evaluate(gbt_test_results)

print('Accuracy of Gradient Boosted Trees Model')
print(acc_gbt)

Accuracy of Gradient Boosted Trees Model
0.9518140377785088